In [1]:
import random
import json
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.decomposition import TruncatedSVD

import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.optim import Adam
from config import *
import os


from collections import Counter

def filter(line):
    #剔除日期
    data_regex = re.compile(u"""        #utf-8编码
        年 |
        月 |
        日 |
        (周一) |
        (周二) | 
        (周三) | 
        (周四) | 
        (周五) | 
        (周六)
    """, re.VERBOSE)
    #剔除所有数字
    decimal_regex = re.compile(r"[^a-zA-Z]\d+")

    line = data_regex.sub(r"", line)
    line = decimal_regex.sub(r"", line)

    return line

def trans(sentence,cutwordslist = None):
    texts_cut = [word for word in jieba.lcut(filter(sentence)) if len(word) > 1]
    outstr = ' '.join(texts_cut)
    return outstr


se = set()
f = open("../input/input.txt", "r", encoding="utf8")
for line in f:
    x = json.loads(line)
    se.add(x["A"])
    se.add(x["B"])
    se.add(x["C"])
    
cutwordslist = []
data = list(se)
for a in range(0, len(data)):
    data[a] = trans(data[a],cutwordslist)

# outputwords = dict(Counter(cutwordslist))
# outputwords_sorted = sorted(outputwords.items(), key= lambda x : x[1], reverse=True)[:100]

# print(outputwords_sorted)

tfidf_model = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b").fit(data)
sparse_result = tfidf_model.transform(data)

f = open("../input/input.txt", "r", encoding="utf8")
ouf = open("../output/output.txt", "w", encoding="utf8")
model = Doc2Vec.load(os.path.join(MODEL_PATH,DOC2VEC))  # you can continue training with the loaded model!


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/8j/p8231m8944s1l43bpl8h5t780000gn/T/jieba.cache
Loading model cost 0.608 seconds.
Prefix dict has been built succesfully.


### lsa

In [2]:
# all_doc = []
# for line in f:
#     x = json.loads(line)
    
#     y = [
#         trans(x["A"]),
#         trans(x["B"]),
#         trans(x["C"])
#     ]

#     y = tfidf_model.transform(y)
#     y = y.todense()
#     all_doc.append(y[0].reshape(-1))
#     all_doc.append(y[1].reshape(-1))
#     all_doc.append(y[2].reshape(-1))
    
# all_doc = np.array(all_doc).reshape(1500,-1)
# print(all_doc.shape)

# # """特征降维：lsa"""
# # print("lsa......")
# # lsa = TruncatedSVD(n_components=100)#TruncatedSVD是SVD的变形，只计算用户指定的最大的K，个奇异值。
# # all_doc = lsa.fit_transform(all_doc)

### doc2vec

In [3]:
f = open("../input/input.txt", "r", encoding="utf8")


X_train = []
for line in f:
    x = json.loads(line)
    y = [
        model.infer_vector(trans(x["A"]).split()),
        model.infer_vector(trans(x["B"]).split()),
        model.infer_vector(trans(x["C"]).split())
    ]
    
    X_train.append(y)
X_train = np.array(X_train)
X_train, X_valid = train_test_split(X_train, test_size = 0.1, random_state = 1996)

In [8]:
print(X_train.shape)

(450, 3, 300)


In [9]:
import torch 
from torch.autograd import Variable 
import torch.nn.functional as F

# 定义一个构建神经网络的类 
class Net(torch.nn.Module): # 继承torch.nn.Module类 
    def __init__(self, n_feature = 4492, n_hidden = 256, n_output = 300): 
        super(Net, self).__init__() # 获得Net类的超类（父类）的构造方法 
        # 定义神经网络的每层结构形式 
        # 各个层的信息都是Net类对象的属性 
        self.hidden1 = torch.nn.Linear(n_feature, 4*n_hidden) # 隐藏层线性输出 
        #self.bn1 = torch.nn.BatchNorm1d(num_features=4*n_hidden)
        self.dropout1 = torch.nn.Dropout(0.8)
        
        self.hidden2 = torch.nn.Linear(4*n_hidden, 4*n_hidden) # 隐藏层线性输出 
        #self.bn2 = torch.nn.BatchNorm1d(num_features=2*n_hidden)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.predict = torch.nn.Linear(4*n_hidden, n_output) # 输出层线性输出 
        self.margin = 1
        
        
    def doc_encoding(self, d):
        d = F.relu(self.hidden1(d))# 对隐藏层的输出进行relu激活 
        d = self.dropout1(d)
        d = F.relu(self.hidden2(d))
        d = self.dropout2(d)
        d = self.predict(d)  # batch_size x 5
        return d

    # 将各层的神经元搭建成完整的神经网络的前向通路 
    def forward(self,d1,d2,d3): 
        d1 = self.doc_encoding(d1)
        
        d2 = self.doc_encoding(d2)
        
        d3 = self.doc_encoding(d3)
        
        pos_sim=F.cosine_similarity(d1, d2)
        neg_sm=F.cosine_similarity(d1, d3)
        
        loss=(self.margin-pos_sim+neg_sm).clamp(min=1e-6).mean()
        return loss

In [10]:
def evaluate_lossAndAcc(net):
    net.eval()
    l_sum ,acc = 0.0 , 0.0
    count = 0
    n=0
    for each in X_valid:
        d1 = each[0]
        d2 = each[1]
        d3 = each[2]
        
        d1 = torch.tensor(d1).view(1,-1).float()
        d2 = torch.tensor(d2).view(1,-1).float()
        d3 = torch.tensor(d3).view(1,-1).float()

        d1_encoding = net.doc_encoding(d1)
        d2_encoding = net.doc_encoding(d2)
        d3_encoding = net.doc_encoding(d3)


        d1_2 = F.cosine_similarity(d1_encoding,d2_encoding)

        d1_3 = F.cosine_similarity(d1_encoding,d3_encoding)

        if d1_2 > d1_3:
            count+=1
        l_sum += net(d1,d2,d3)
        n+=1
    net.train()
    return l_sum / n, count / X_valid.shape[0]

In [11]:
def save_model(network, path = MODEL_PATH, name=MODEL_NAME):
#     torch.save(network, os.path.join(path, name))
    state = network.state_dict()
    for key in state:
        state[key] = state[key].clone().cpu()
    torch.save(network.state_dict(),os.path.join(path,name))

In [12]:
net = Net(X_train.shape[2],256,300)
# 定义优化器和损失函数 
optimizer = Adam(net.parameters(), lr=0.001, weight_decay=1e-4)


best_loss = 100
for t in range(100):
    train_lsum, n , acc = 0.0, 0, 0.0
    count = 0
    for each in tqdm(X_train):
        d1 = each[0]
        d2 = each[1]
        d3 = each[2]
        d1 = torch.tensor(d1).view(1,-1).float()
        d2 = torch.tensor(d2).view(1,-1).float()
        d3 = torch.tensor(d3).view(1,-1).float()
        
        
        
        
        d1_encoding = net.doc_encoding(d1)
        d2_encoding = net.doc_encoding(d2)
        d3_encoding = net.doc_encoding(d3)


        d1_2 = F.cosine_similarity(d1_encoding,d2_encoding)

        d1_3 = F.cosine_similarity(d1_encoding,d3_encoding)

        if d1_2 > d1_3:
            count+=1
        
        
        optimizer.zero_grad()
        loss = net(d1,d2,d3)
        train_lsum += loss
        loss.backward()
        optimizer.step()
        n+=1
    loss,acc = evaluate_lossAndAcc(net)
    print("epoch %d | train_loss : %.4f | train_acc : %.4f | valid loss : %.4f | valid_acc : %.4f" 
                       % (t, train_lsum/n,count / X_train.shape[0],loss, acc))
    
    if loss < best_loss:
        best_loss = loss
        print('save the current best model | loss : %.4f | valid acc : %.4f' % (best_loss.item(), acc))
        save_model(net)

  1%|          | 5/450 [00:00<00:09, 48.58it/s]

epoch 0 | train_loss : 0.9831 | train_acc : 0.5644 | valid loss : 0.9267 | valid_acc : 0.7000
save the current best model | loss : 0.9267 | valid acc : 0.7000


  1%|          | 5/450 [00:00<00:09, 46.61it/s]

epoch 1 | train_loss : 0.9451 | train_acc : 0.5644 | valid loss : 0.9686 | valid_acc : 0.6600


  1%|          | 5/450 [00:00<00:10, 42.52it/s]

...

epoch 28 | train_loss : 0.4572 | train_acc : 0.8067 | valid loss : 0.5326 | valid_acc : 0.8200


  0%|          | 2/450 [00:00<00:23, 19.33it/s]

epoch 29 | train_loss : 0.4510 | train_acc : 0.8422 | valid loss : 0.5398 | valid_acc : 0.8000


 14%|█▎        | 61/450 [00:03<00:20, 19.15it/s]


KeyboardInterrupt: 

In [14]:
count= 0 
net.eval()
for each in tqdm(X_valid):
    d1 = each[0]
    d2 = each[1]
    d3 = each[2]
    
    d1 = torch.tensor(d1).view(1,-1).float()
    d2 = torch.tensor(d2).view(1,-1).float()
    d3 = torch.tensor(d3).view(1,-1).float()


    d1_encoding = net.doc_encoding(d1)
    d2_encoding = net.doc_encoding(d2)
    d3_encoding = net.doc_encoding(d3)


    d1_2 = F.cosine_similarity(d1_encoding,d2_encoding)

    d1_3 = F.cosine_similarity(d1_encoding,d3_encoding)
    
    
    
    if d1_2 > d1_3:
        count+=1
print('Acc :', count/X_valid.shape[0])

100%|██████████| 50/50 [00:00<00:00, 439.68it/s]

Acc : 0.8


In [16]:
model_dict = torch.load(os.path.join(MODEL_PATH, MODEL_NAME))
net = Net(X_train.shape[2],256,300)

net.load_state_dict(model_dict)
net.eval()


X_all = np.vstack((X_train, X_valid))


test_y = []

count= 0 
for each in tqdm(X_valid):
    d1 = each[0]
    d2 = each[1]
    d3 = each[2]
    
    d1 = torch.tensor(d1).view(1,-1).float()
    d2 = torch.tensor(d2).view(1,-1).float()
    d3 = torch.tensor(d3).view(1,-1).float()


    d1_encoding = net.doc_encoding(d1)
    d2_encoding = net.doc_encoding(d2)
    d3_encoding = net.doc_encoding(d3)

    d1_2 = F.cosine_similarity(d1_encoding,d2_encoding)

    d1_3 = F.cosine_similarity(d1_encoding,d3_encoding)
    
    if d1_2 > d1_3:
        count+=1
        
print('X_train :', X_valid.shape[0])
print('B acc :', count/X_valid.shape[0])

100%|██████████| 50/50 [00:00<00:00, 717.45it/s]

X_train : 50
B acc : 0.84
